<a href="https://colab.research.google.com/github/manmustbecool/Experiment/blob/main/llm_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import torch

# Step 1: Load the smallest LLM and tokenizer
# Using "bigscience/bloom-560m" as an example of a small LLM.
model_name = "bigscience/bloom-560m"
# model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Step 2: Configure PEFT with LoRA (Low-Rank Adaptation)
# LoRA reduces the number of trainable parameters, making fine-tuning efficient.
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # Specify the task type (causal language modeling)
    inference_mode=False,          # Set to False for training mode
    r=4,                           # Rank of the LoRA matrices (smaller for efficiency)
    lora_alpha=16,                 # Scaling factor for LoRA
    lora_dropout=0.1               # Dropout rate for LoRA layers
)
model = get_peft_model(model, peft_config)  # Wrap the base model with the PEFT configuration


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

ValueError: Unrecognized configuration class <class 'transformers.models.t5.configuration_t5.T5Config'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of AriaTextConfig, BambaConfig, BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CohereConfig, Cohere2Config, CpmAntConfig, CTRLConfig, Data2VecTextConfig, DbrxConfig, DeepseekV3Config, DiffLlamaConfig, ElectraConfig, Emu3Config, ErnieConfig, FalconConfig, FalconMambaConfig, FuyuConfig, GemmaConfig, Gemma2Config, Gemma3Config, Gemma3TextConfig, GitConfig, GlmConfig, Glm4Config, GotOcr2Config, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, GraniteConfig, GraniteMoeConfig, GraniteMoeSharedConfig, HeliumConfig, JambaConfig, JetMoeConfig, LlamaConfig, Llama4Config, Llama4TextConfig, MambaConfig, Mamba2Config, Ma

In [8]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
dataset = load_dataset("imdb")
print(dataset['train'])

In [ ]:
# Step 3: Load the IMDb dataset and create a small sample
dataset = load_dataset("imdb", split="train")  # Load the full training split
subset_size = int(0.005 * len(dataset))  # Calculate 0.5% of the dataset size
small_sample = dataset.select(range(subset_size))  # select a subset
print(small_sample)

# Step 4: Tokenize the dataset
# Convert text data into tokenized format suitable for the model.
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=512  # Align batch size to match your training configuration
)
print(tokenized_dataset)

print(tokenized_dataset[0])

In [ ]:
# Step 5: Define training arguments
# Specify hyperparameters and settings for the training process.
training_args = TrainingArguments(
    output_dir="./results",          # Directory to save training results
    # eval_strategy="epoch",    # Evaluate the model at the end of each epoch
    learning_rate=2e-5,             # Learning rate for the optimizer
    per_device_train_batch_size=4,  # Batch size per device
    num_train_epochs=1,             # Number of training epochs
    weight_decay=0.01,              # Weight decay for regularization
    save_total_limit=1,             # Limit the number of saved checkpoints
    label_names=["label"],          # Explicitly set label_names
    report_to="none"                # Disable integration with W&B
)

# Step 6: Initialize the Trainer
# The Trainer class handles the training loop and evaluation.
trainer = Trainer(
    model=model,                    # Model to be trained
    args=training_args,             # Training arguments
    train_dataset=tokenized_dataset # Training dataset
)

# Step 7: Fine-tune the model
trainer.train()

# Step 8: Save the fine-tuned model
# Save the model and tokenizer for future use.
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

print("training finished")

In [ ]:
# Import necessary libraries
from datasets import Dataset
from sklearn.metrics import accuracy_score
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import torch

# Sample dataset provided in the prompt
sample_data = [
    {"prompt": "ww", "response": "ssss"},
    {"prompt": "dd", "response": "ss"},
    {"prompt": "ss", "response": "sss"}
]

# Load the dataset into a Hugging Face Dataset object
dataset = Dataset.from_list(sample_data)

# Initialize the tokenizer and the small LLM (e.g., GPT-2-small)
model_name = "bigscience/bloom-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Step 2: Configure PEFT with LoRA (Low-Rank Adaptation)
# LoRA reduces the number of trainable parameters, making fine-tuning efficient.
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # Specify the task type (causal language modeling)
    inference_mode=False,          # Set to False for training mode
    r=4,                           # Rank of the LoRA matrices (smaller for efficiency)
    lora_alpha=16,                 # Scaling factor for LoRA
    lora_dropout=0.1               # Dropout rate for LoRA layers
)
# model = get_peft_model(model, peft_config)  # Wrap the base model with the PEFT configuration


# Tokenize the sample dataset for fine-tuning
def preprocess_function(example):
    inputs = tokenizer(example["prompt"], truncation=True, padding=True, max_length=64, return_tensors="pt")
    labels = tokenizer(example["response"], truncation=True, padding=True, max_length=64, return_tensors="pt")["input_ids"]
    return {"input_ids": inputs["input_ids"][0], "labels": labels[0]}

tokenized_dataset = dataset.map(preprocess_function)

for i in range(len(tokenized_dataset)):
    print(tokenized_dataset[i])

# Step 5: Define training arguments
# Specify hyperparameters and settings for the training process.
training_args = TrainingArguments(
    output_dir="./results",          # Directory to save training results
    # eval_strategy="epoch",    # Evaluate the model at the end of each epoch
    learning_rate=2e-5,             # Learning rate for the optimizer
    per_device_train_batch_size=8,  # Batch size per device
    num_train_epochs=1,             # Number of training epochs
    weight_decay=0.01,              # Weight decay for regularization
    save_total_limit=1,             # Limit the number of saved checkpoints
    label_names=["labels"],          # Explicitly set label_names
    report_to="none"                # Disable integration with W&B
)

# Step 6: Initialize the Trainer
# The Trainer class handles the training loop and evaluation.
trainer = Trainer(
    model=model,                    # Model to be trained
    args=training_args,             # Training arguments
    train_dataset=tokenized_dataset # Training dataset
)

# Step 7: Fine-tune the model
trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

ValueError: expected sequence of length 2 at dim 1 (got 1)

In [ ]:
# Step 5: Define training arguments
# Specify hyperparameters and settings for the training process.
training_args = TrainingArguments(
    output_dir="./results",          # Directory to save training results
    evaluation_strategy="epoch",    # Evaluate the model at the end of each epoch
    learning_rate=2e-5,             # Learning rate for the optimizer
    per_device_train_batch_size=8,  # Batch size per device
    num_train_epochs=1,             # Number of training epochs
    weight_decay=0.01,              # Weight decay for regularization
    save_total_limit=1,             # Limit the number of saved checkpoints
)

# Step 6: Initialize the Trainer
# The Trainer class handles the training loop and evaluation.
trainer = Trainer(
    model=model,                    # Model to be trained
    args=training_args,             # Training arguments
    train_dataset=tokenized_dataset # Training dataset
)

# Step 7: Fine-tune the model
trainer.train()

# Step 8: Save the fine-tuned model
# Save the model and tokenizer for future use.
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

# Step 9: Compare the fine-tuned model with the original model
# Evaluate both models on the same dataset and compare their losses.
def evaluate_model(model, tokenizer, dataset):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    for example in dataset:
        inputs = tokenizer(example["text"], return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        with torch.no_grad():  # Disable gradient computation for evaluation
            outputs = model(**inputs, labels=inputs["input_ids"])  # Compute loss
        total_loss += outputs.loss.item()  # Accumulate loss
    return total_loss / len(dataset)  # Return average loss

# Load the original model for comparison
original_model = AutoModelForCausalLM.from_pretrained(model_name)

# Evaluate both models
original_loss = evaluate_model(original_model, tokenizer, tokenized_dataset)
fine_tuned_loss = evaluate_model(model, tokenizer, tokenized_dataset)

# Print the comparison results
print(f"Original Model Loss: {original_loss}")
print(f"Fine-Tuned Model Loss: {fine_tuned_loss}")

In [22]:
## test

# Import necessary libraries
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
import torch


# Initialize tokenizer and model
model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


# Sample dataset
sample_data = [
    {"prompt": "wer?", "response": "no"},
    {"prompt": "wer wer?", "response": "no no no"},
]*300

# Load dataset
dataset = Dataset.from_list(sample_data)

# Tokenization function with uniform padding
def preprocess_function(example):
    inputs = tokenizer(example["prompt"], truncation=True, padding="max_length", max_length=64, return_tensors="pt")
    labels = tokenizer(example["response"], truncation=True, padding="max_length", max_length=64, return_tensors="pt")["input_ids"]
    return {"input_ids": inputs["input_ids"].squeeze(0), "labels": labels.squeeze(0)}

tokenized_dataset = dataset.map(preprocess_function)

# Define data collator for consistent tensor shapes
data_collator = DataCollatorWithPadding(tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=30,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=1,
    label_names=["labels"],
    report_to="none"
)

# Initialize Trainer with data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator  # Handles padding dynamically
)

# Fine-tune the model
trainer.train()

model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

Model Output: wer?
I'm not sure if it


Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Step,Training Loss


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [25]:
# Initialize tokenizer and model
model_name = "./fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Test the model with a sample prompt
input_text = "wer?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(inputs["input_ids"], max_length=20) # Generate response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Model Output:", response)

Model Output: wer?

wer

wer

wer

wer

wer




In [24]:
# Initialize tokenizer and model
model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Test the model with a sample prompt
input_text = "wer?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(inputs["input_ids"], max_length=20) # Generate response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Model Output:", response)

Model Output: wer?
I'm not sure if it's a joke or not, but I'm pretty
